In [1]:
from vnstock.api.listing import Listing
from vnstock.api.company import Company
from vnstock.api.financial import Finance
from vnstock.api.quote import Quote
import pandas as pd
from typing import Callable
import warnings
import time

warnings.filterwarnings("ignore")

In [2]:
listing = Listing(source="VCI")
vn30_symbols = listing.symbols_by_group("VN30").to_list()

In [3]:
df_company = listing.symbols_by_industries()
df_company = df_company[df_company["symbol"].isin(vn30_symbols)].reset_index(drop=True)

company_cols = ["symbol", "organ_name", "icb_code1", "icb_code2", "icb_code3", "icb_code4"]
df_company = df_company[company_cols]
df_company.head(5)

,symbol,organ_name,icb_code1,icb_code2,icb_code3,icb_code4
0,BID,Ngân hàng Thương mại Cổ phần Đầu tư và Phát tr...,8301,8300,8350,8355
1,BCM,Tập đoàn Đầu tư và Phát triển Công nghiệp Beca...,8000,8600,8630,8633
2,DGC,Công ty Cổ phần Tập đoàn Hóa chất Đức Giang,1000,1300,1350,1357
3,FPT,Công ty Cổ phần FPT,9000,9500,9530,9537
4,GAS,Tổng Công ty Khí Việt Nam - Công ty Cổ phần,7000,7500,7570,7573


In [4]:
df_industry = listing.industries_icb()

industry_cols = ["icb_code", "level", "icb_name", "en_icb_name"]
df_industry = df_industry[industry_cols]

df_industry

,icb_code,level,icb_name,en_icb_name
0,0530,3,Sản xuất Dầu khí,Oil & Gas Producers
1,0570,3,"Thiết bị, Dịch vụ và Phân phối Dầu khí","Oil Equipment, Services & Distribution"
2,1350,3,Hóa chất,Chemicals
3,1730,3,Lâm nghiệp và Giấy,Forestry & Paper
4,1750,3,Kim loại,Industrial Metals & Mining
...,...,...,...,...
150,6000,1,Viễn thông,Telecommunications
151,7000,1,Tiện ích Cộng đồng,Utilities
152,8000,1,Tài chính,Financials
153,8301,1,Ngân hàng,Banks


In [5]:
def get_company_objects(symbols: list[str], source: str = "VCI") -> list[Company]:
    company_objects = []
    for s in symbols:
        company_objects.append(Company(symbol=s, source=source))
    return company_objects

# Run
vn30_companies = get_company_objects(vn30_symbols, "VCI")

In [6]:
def get_company_info(func: Callable[[Company], pd.DataFrame], company_objects: list[Company], **kwargs) -> pd.DataFrame:
    dfs = []
    for c in company_objects:
        df = func(c, **kwargs)
        df["symbol"] = c.symbol
        dfs.append(df)

    dfs = pd.concat(dfs, axis=0, ignore_index=True)
    return dfs

In [7]:
time.sleep(60)
df_company_shareholders = get_company_info(Company.shareholders, vn30_companies)
df_company_shareholders.head(5)

,id,share_holder,quantity,share_own_percent,update_date,symbol
0,92584797,Vietnam Enterprise Investments Limited,212880184,0.054809,2024-09-13,ACB
1,92579608,Sather Gate Investments Limited,193907186,0.049900,2025-10-05,ACB
2,92560864,Estes Investments Limited,83010435,0.049900,2024-05-15,ACB
3,92577372,Phạm Thị Thu Hà,285000,0.049900,2016-05-20,ACB
4,92560444,Dragon Financial Holdings Limited,140770684,0.036243,2025-10-05,ACB


In [8]:
df_company_officers = get_company_info(Company.officers, vn30_companies)
df_company_officers.head(5)

,id,officer_name,officer_position,position_short_name,update_date,officer_own_percent,quantity,symbol
0,6,Trần Hùng Huy,Chủ tịch Hội đồng Quản trị,Chủ tịch HĐQT,2025-08-12,0.0343,176021482,ACB
1,7,Đặng Thu Thủy,Thành viên Hội đồng Quản trị,TV HĐQT,2025-08-12,0.0119,61352541,ACB
2,17,Đặng Phú Vinh,Giám đốc khối,GĐ Khối,2025-08-12,0.0037,18922683,ACB
3,8,Đỗ Minh Toàn,Tổng Giám đốc,TGĐ,2025-08-12,0.0007,3683318,ACB
4,9,Nguyễn Thành Long,Phó Chủ tịch Hội đồng Quản trị,Phó Chủ tịch HĐQT,2025-08-12,0.0004,1894127,ACB


In [9]:
time.sleep(60)
df_company_news = get_company_info(Company.news, vn30_companies)

with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(df_company_news.head(1))

,id,news_title,news_sub_title,friendly_sub_title,news_image_url,news_source_link,created_at,public_date,updated_at,lang_code,news_id,news_short_content,news_full_content,close_price,ref_price,floor,ceiling,price_change_pct,symbol
0,8817127,ACB: Quyết định của NHNNVN về việc bổ sung nội dung hoạt động vào Giấy phép thành lập và hoạt động,,,https://cdn.fiingroup.vn/medialib/127889/I/2024/11/25/16373468562400700_ACB1.png,https://www.hsx.vn/vi/tin-tuc/acb-quyet-dinh-cua-nhnnvn-ve-viec-bo-sung-noi-dung-hoat-dong-vao-giay-phep-thanh-lap-va-hoat-dong/2418034,None,1763135855000,None,vi,11793545,Ngân hàng Thương mại Cổ phần Á Châu công bố Quyết định của NHNNVN về việc bổ sung nội dung hoạt động vào Giấy phép thành lập và hoạt động như sau:,"<p>Ngân hàng Thương mại Cổ phần Á Châu công bố Quyết định của NHNNVN về việc bổ sung nội dung hoạt động vào Giấy phép thành lập và hoạt động như sau:</p><table width=""100%"" style='text-align: left;border=0'><tr><td colspan='2'><hr /></td></tr><tr><td colspan='2'>Tài liệu đính kèm</td></tr><tr><td> </td><td><a href=""https://cmsv5.fiingroup.vn/medialib/FG/2025/2025-11/2025-11-14/ACB/20251113--ACB--TB-thay-doi-Giay-phep.pdf"" title=""Tải về"" download>20251113--ACB--TB-thay-doi-Giay-phep.pdf</a></td></tr></table>",24950.0,25100.0,23350.0,26850.0,-0.005976,ACB


In [10]:
df_company_events = get_company_info(Company.events, vn30_companies)

with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(df_company_events.head(1))

,id,event_title,en__event_title,public_date,issue_date,source_url,event_list_code,ratio,value,record_date,exright_date,event_list_name,en__event_list_name,symbol
0,18135,Trả cổ tức bằng tiền mặt,,2013-05-10,2013-06-04,,DIV,NaN,NaN,2013-05-21,2013-05-17,Trả cổ tức bằng tiền mặt,Cash Dividend,ACB


In [11]:
def get_quote_objects(symbols: list[str], source: str = "VCI") -> list[Quote]:
    quote_objects = []
    for s in symbols:
        quote_objects.append(Quote(symbol=s, source=source))
    return quote_objects

# Run
vn30_quotes = get_quote_objects(vn30_symbols, "VCI")

In [12]:
def get_ohlcv(func: Callable[[Quote], pd.DataFrame], quote_objects: list[Quote], **kwargs) -> pd.DataFrame:
    dfs = []
    for q in quote_objects:
        df = func(q, **kwargs)
        df["symbol"] = q.symbol
        dfs.append(df)

    dfs = pd.concat(dfs, axis=0, ignore_index=True)
    cols = ["symbol", "time", "open", "high", "low", "close", "volume"]
    dfs = dfs[cols].sort_values(by=["time"], ascending=[True]).reset_index(drop=True)

    return dfs

In [13]:
time.sleep(60)

ohlcv_1d_args = {"start": "2022-01-01", "interval": "1D"}
df_ohlcv_1d = get_ohlcv(Quote.history, vn30_quotes, **ohlcv_1d_args)
df_ohlcv_1d

,symbol,time,open,high,low,close,volume
0,ACB,2021-11-12,15.48,15.64,15.31,15.64,4376100
1,VNM,2021-11-12,70.51,70.59,69.81,69.96,2120400
2,VJC,2021-11-12,128.20,128.90,127.50,128.60,755200
3,VIC,2021-11-12,94.80,94.80,93.90,94.50,1133100
4,VIB,2021-11-12,14.89,15.07,14.54,15.05,988000
...,...,...,...,...,...,...,...
29995,VIB,2025-11-14,18.50,18.65,18.45,18.50,2465600
29996,VIC,2025-11-14,210.80,211.30,208.10,211.00,2083100
29997,VJC,2025-11-14,175.60,177.00,173.30,176.40,1849300
29998,MWG,2025-11-14,79.20,82.00,79.20,81.40,3951700


In [14]:
def get_finance_objects(symbols: list[str], source: str = "VCI") -> list[Finance]:
    finance_objects = []
    for s in symbols:
        finance_objects.append(Finance(symbol=s, source=source))
    return finance_objects

# Run
vn30_finances = get_finance_objects(vn30_symbols, "VCI")

In [15]:
def get_finance_ratios(func: Callable[[Finance], pd.DataFrame], finance_objects: list[Finance], **kwargs) -> pd.DataFrame:
    dfs = []
    for f in finance_objects:
        df = func(f, **kwargs)
        dfs.append(df)

    dfs = pd.concat(dfs, axis=0, ignore_index=True)
    if ("Meta", "yearReport") in dfs.columns:
        dfs = dfs[dfs[("Meta", "yearReport")] >= 2020]
    dfs = dfs.sort_values(by=("Meta", "yearReport"), ascending=True).reset_index(drop=True)
    return dfs

In [16]:
time.sleep(60)
finance_args = {"period": "year", "lang": "en"}
df_ratio_by_year = get_finance_ratios(Finance.ratio, vn30_finances, **finance_args)
df_ratio_by_year

Meta                         Chỉ tiêu cơ cấu nguồn vốn  \
    ticker yearReport lengthReport     Fixed Asset-To-Equity   
0      ACB       2020            5                  0.106712   
1      BCM       2020            5                  0.113407   
2      BID       2020            5                  0.130855   
3      CTG       2020            5                  0.126698   
4      DGC       2020            5                  0.498955   
..     ...        ...          ...                       ...   
145    FPT       2024            5                  0.414698   
146    VJC       2024            5                  0.827529   
147    VNM       2024            5                  0.346946   
148    VPB       2024            5                  0.013742   
149    VRE       2024            5                  0.005281   

                                   Chỉ tiêu khả năng sinh lợi            \
    Owners' Equity/Charter Capital      Net Profit Margin (%)   ROE (%)   
0                         0.690102                   0.526866  0.243075   
1                         1.601698                   0.322476  0.120564   
2                         1.134347                   0.195454  0.091845   
3                         1.591049                   0.385620  0.169043   
4                         1.070999                   0.145382  0.241169   
..                             ...                        ...       ...   
145                       2.097293                   0.125011  0.286909   
146                       2.893574                   0.019476  0.086807   
147                       1.730870                   0.152022  0.293621   
148                       1.856273                   0.321489  0.111379   
149                       1.800168                   0.458186  0.102747   

                                 Chỉ tiêu thanh khoản  ...  \
      ROA (%) Dividend yield (%)   Financial Leverage  ...   
0    0.018557           0.000000            12.540286  ...   
1    0.043164           0.012500             2.962832  ...   
2    0.004805           0.005391            19.042690  ...   
3    0.010656           0.023599            15.701336  ...   
4    0.171103           0.009381             1.444683  ...   
..        ...                ...                  ...  ...   
145  0.118787           0.020000             2.015252  ...   
146  0.015068           0.000000             5.801561  ...   
147  0.174380           0.069268             1.521768  ...   
148  0.018361           0.017986             6.272932  ...   
149  0.079623           0.000000             1.317335  ...   

    Chỉ tiêu khả năng sinh lợi                                    \
               EBIT Margin (%) Gross Profit Margin (%)  ROIC (%)   
0                          NaN                     NaN       NaN   
1                     0.293493                0.501210  0.059464   
2                          NaN                     NaN       NaN   
3                          NaN                     NaN       NaN   
4                     0.158189                0.237183  0.189123   
..                         ...                     ...       ...   
145                   0.167200                0.377069  0.207368   
146                   0.043871                0.099289  0.053004   
147                   0.168406                0.414197  0.228934   
148                        NaN                     NaN       NaN   
149                   0.422033                0.531976  0.081209   

                                    Chỉ tiêu thanh khoản             \
    EBITDA (Bn. VND) EBIT (Bn. VND)        Current Ratio Cash Ratio   
0                NaN            NaN                  NaN        NaN   
1       2.145587e+12   1.909650e+12             1.390992   0.082767   
2                NaN            NaN                  NaN        NaN   
3                NaN            NaN                  NaN        NaN   
4       1.254348e+12   9.865402e+11             1.898409   0.156097   
..      

In [18]:
time.sleep(60)
finance_args = {"period": "quarter", "lang": "en"}
df_ratio_by_quarter = get_finance_ratios(Finance.ratio, vn30_finances, **finance_args)
df_ratio_by_quarter

Meta                         Chỉ tiêu cơ cấu nguồn vốn  \
    ticker yearReport lengthReport               Debt/Equity   
0      VRE       2020            1                  0.345040   
1      ACB       2020            4                 11.540286   
2      ACB       2020            2                 11.868276   
3      ACB       2020            3                 11.720578   
4      ACB       2020            1                 12.171197   
..     ...        ...          ...                       ...   
685    MBB       2025            3                  8.982364   
686    MBB       2025            2                  9.090825   
687    MBB       2025            1                  8.354378   
688    ACB       2025            2                  9.704445   
689    ACB       2025            1                  9.240355   

                                                          \
    Fixed Asset-To-Equity Owners' Equity/Charter Capital   
0                0.017116                       1.178525   
1                0.106712                       0.690102   
2                0.121825                       0.600243   
3                0.113291                       0.640864   
4                0.128266                       0.572597   
..                    ...                            ...   
685              0.038478                       1.652275   
686              0.041809                       1.586638   
687              0.043515                       1.535401   
688              0.061880                       1.697809   
689              0.061005                       1.695161   

    Chỉ tiêu khả năng sinh lợi                                ...  \
         Net Profit Margin (%)   ROE (%)  ROIC (%)   ROA (%)  ...   
0                     0.291923  0.099448  0.113418  0.073963  ...   
1                     0.577377  0.243075  0.000000  0.018557  ...   
2                     0.488972  0.217257  0.000000  0.016141  ...   
3                     0.570721  0.221063  0.000000  0.016849  ...   
4                     0.449504  0.231970  0.000000  0.016671  ...   
..                         ...       ...       ...       ...  ...   
685                   0.428853  0.193772  0.000000  0.019821  ...   
686                   0.475069  0.204944  0.000000  0.021302  ...   
687                   0.561721  0.216881  0.000000  0.022931  ...   
688                   0.730292  0.201725  0.000000  0.019587  ...   
689                   0.578447  0.204361  0.000000  0.020059  ...   

    Chỉ tiêu hiệu quả hoạt động Chỉ tiêu khả năng sinh lợi  \
             Inventory Turnover            EBIT Margin (%)   
0                      4.803106                   0.360599   
1                           NaN                        NaN   
2                           NaN                        NaN   
3                           NaN                        NaN   
4                           NaN                        NaN   
..                          ...                        ...   
685                         NaN                        NaN   
686                         NaN                        NaN   
687                         NaN                        NaN   
688                         NaN                        NaN   
689                         NaN                        NaN   

                                                             \
    Gross Profit Margin (%) EBITDA (Bn. VND) EBIT (Bn. VND)   
0                  0.450214     4.894577e+12   6.078500e+11   
1                       NaN              NaN            NaN   
2                       NaN              NaN            NaN   
3                       NaN              NaN            NaN   
4                       NaN              NaN            NaN   
..                      ...              ...            ...   
685                     NaN              NaN            NaN   
686                     NaN              NaN            NaN   
687                     NaN              NaN            N

In [19]:
df_ratio_by_year.columns = [col[1] if col[1] != "" else col[0] for col in df_ratio_by_year.columns]
df_ratio_by_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 37 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ticker                           150 non-null    object 
 1   yearReport                       150 non-null    int64  
 2   lengthReport                     150 non-null    int64  
 3   Fixed Asset-To-Equity            150 non-null    float64
 4   Owners' Equity/Charter Capital   150 non-null    float64
 5   Net Profit Margin (%)            150 non-null    float64
 6   ROE (%)                          150 non-null    float64
 7   ROA (%)                          150 non-null    float64
 8   Dividend yield (%)               145 non-null    float64
 9   Financial Leverage               150 non-null    float64
 10  Market Capital (Bn. VND)         150 non-null    float64
 11  Outstanding Share (Mil. Shares)  150 non-null    float64
 12  P/E                   

In [20]:
df_ratio_by_quarter.columns = [col[1] if col[1] != "" else col[0] for col in df_ratio_by_quarter.columns]
df_ratio_by_quarter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 37 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ticker                           690 non-null    object 
 1   yearReport                       690 non-null    int64  
 2   lengthReport                     690 non-null    int64  
 3   Debt/Equity                      690 non-null    float64
 4   Fixed Asset-To-Equity            690 non-null    float64
 5   Owners' Equity/Charter Capital   690 non-null    float64
 6   Net Profit Margin (%)            690 non-null    float64
 7   ROE (%)                          690 non-null    float64
 8   ROIC (%)                         529 non-null    float64
 9   ROA (%)                          690 non-null    float64
 10  Dividend yield (%)               552 non-null    float64
 11  Financial Leverage               690 non-null    float64
 12  Market Capital (Bn. VN

In [21]:
df_ratio_by_quarter.lengthReport.value_counts()

lengthReport
1    180
2    180
3    180
4    150
Name: count, dtype: int64